In [ ]:
# !pip install purecpp_extract purecpp_chunks_clean

In [ ]:
# !git clone https://github.com/PureAI-Tools/Tests.git # comentar isso no jupiter local 
API_KEY = ""

In [ ]:
import purecpp_extract as ext
import purecpp_chunks_clean as cc
from purecpp_libs import DataExtractRequestStruct, LoaderDataStruct, RAGDocument

In [ ]:
pdf_loader = ext.PDFLoader()
pdf_loader2 = ext.PDFLoader()
pdf_loader3 = ext.PDFLoader()

In [ ]:
docxLoader = ext.DOCXLoader()
docxLoader2 = ext.DOCXLoader()

In [ ]:
var = pdf_loader.InsertDataToExtract([
        DataExtractRequestStruct("pdfs/1984.pdf"),
        DataExtractRequestStruct("pdfs/a-christmas-carol.pdf"),
    ]) # No jupiter ta imprimindo q tao sendo achados arquivos aqui n
print(f"Content : \n {var}") # Poderia...

In [ ]:
# pdf_loader2.InsertDataToExtract([DataExtractRequestStruct("pdfs")]) # deveria dar erro

In [ ]:
pdf_loader3.InsertDataToExtract([DataExtractRequestStruct("pdfs/1984.pdf")])

In [ ]:
docxLoader.InsertDataToExtract([
        DataExtractRequestStruct("docxs/sample1.docx"),
        DataExtractRequestStruct("docxs/sample4.docx")
    ]) # Tem que avisar que n encontra a pasta ou o arquivo na pasta se n o encontrar

In [ ]:
docxLoader2.InsertDataToExtract([DataExtractRequestStruct("docxs")])

In [ ]:
opt_struct = pdf_loader.GetTextContent("1984") # N retorna um tipo string
text_content = str(opt_struct)
text_content_limited = text_content[:100]
# print(text_content_limited)
for i in range(3):
  print(opt_struct.textContent[i:i+1])
  print("\n")

print(f"{len(opt_struct.textContent)} Pages")# Isso salva como paginas



In [ ]:
print(type(opt_struct))
for item in dir(opt_struct):
    print(item)
print(type(dir(opt_struct)))
# help(opt_struct)
type(opt_struct.textContent)

In [ ]:
for attr_name in dir(opt_struct):
    if not attr_name.startswith("__"):  # Ignora atributos internos
        try:
            value = getattr(opt_struct, attr_name)
            if attr_name == "textContent":
                print(f"{attr_name}: Is a list of {len(value)} Pages")
                print(f"Page 0 {attr_name} :\n{value[0]} ")
            else:
              print(f"{attr_name}: {value}")
            print("----------------------------------------------")
        except AttributeError:
            print(f"{attr_name}: [não acessível]")

In [ ]:
pdf_loader.GetTextContent("1984")

In [ ]:
if not pdf_loader3.GetTextContent("a-christmas-carol"): # DEVERIA DAR ERRO
    print("deu erro")

In [ ]:
pdf_loader3.GetTextContent("sdauhdsahudsauhdsahuds") # DEVERIA DAR ERRO

In [ ]:
# print(type(pdf_loader))
# for item in dir(pdf_loader):
#     print(item)
# print(type(dir(pdf_loader)))

In [ ]:
# help(pdf_loader)

In [ ]:
items_docx = []

for fname in ["sample1", "sample4"]: # poderia ter um metodo para recuperar as listas naquela List de LoaderDataStruct < --------------- >
    x = docxLoader.GetTextContent(fname)
    if x:
        items_docx.append(x.textContent)  # cada item é um List em LoaderDataStruct
    else:
        print(f"Não foi possível extrair: {fname}")
if not items_docx:
        print("Nenhum documento foi carregado com sucesso! Encerrando.")
print(f"Total de documentos carregados: {len(items_docx)}\n")

In [ ]:
items = []
item_struct =  []

for fname in ["1984", "a-christmas-carol"]: # poderia ter um metodo para recuperar as listas naquela List de LoaderDataStruct < --------------- >
    opt_struct = pdf_loader.GetTextContent(fname)
    if opt_struct:
        items.append(opt_struct.textContent)   # cada item é um List em LoaderDataStruct
        item_struct.append(opt_struct) # cada item é LoaderDataStruct
    else:
        print(f"Não foi possível extrair: {fname}")
if not items:
        print("Nenhum documento foi carregado com sucesso! Encerrando.")
print(f"Total de documentos carregados: {len(items)}\n")

In [ ]:
print("---------------------------------------------------------")
for i, lds in enumerate(item_struct):
    print(f"--- LoaderDataStruct nº {i} ---")

    print(">> METADATA:")
    for mk, mv in lds.metadata.items():
        print(f"   {mk} => {mv}")

    print(f"\n>> TEXT CONTENT=> {lds.textContent[10][:100]} ...")
    print()
print("---------------------------------------------------------")

In [ ]:
# 2) CHUNKANDO COM ChunkDefault
#    (divide texto em blocos de 300 chars, com overlap=50)
# default_chunker = cc.ChunkDefault(chunk_size=300, overlap=50)
default_chunker = cc.ChunkDefault(50, 10) # poderia passar o item ja aqui
default_chunker2 = cc.ChunkDefault(50, 10)

In [ ]:
# std::vector<RAGLibrary::Document> ProcessSingleDocument(RAGLibrary::LoaderDataStruct &item);
default_docs = default_chunker.ProcessData(item_struct[0])
# std::vector<RAGLibrary::Document> ProcessDocuments(const std::vector<RAGLibrary::LoaderDataStruct> &items, int max_workers = 4);
default_docs2 = default_chunker2.ProcessData(item_struct, max_workers=2) # Tem q ser uma lista de purecpp_libs.purecpp_libs.LoaderDataStruct

In [ ]:
print(f"ChunkDefault(50, 10) from  documents.size={len(default_docs)}")
print(f"ChunkDefault(50, 10) from  documents.size={len(default_docs2)}")

In [ ]:
# print(type(default_docs[0])) # purecpp_libs.purecpp_libs.RAGDocument
# for item in dir(default_docs[0]):
#     print(item)

In [ ]:
# default_docs[0]
# help(default_docs)

In [ ]:

print("-----------------------------------------------------------------------------------------")
print("default")
list_default_docs = []
# Garantir que temos ao menos 3 elementos
n = len(default_docs)
first_n = default_docs[:3]            # primeiros 3
last_n = default_docs[-3:] if n >= 3 else []  # últimos 3 (sem repetir os primeiros)
# Adiciona os page_content dos primeiros
for i, doc in enumerate(first_n):
    print(f"Primeiro [{i}] METADATA =>", doc.metadata)
    list_default_docs.append(doc.page_content)
# Adiciona os page_content dos últimos
for i, doc in enumerate(last_n):
    print(f"Último [{n - 3 + i}] METADATA =>", doc.metadata)
    list_default_docs.append(doc.page_content)

print("-----------------------------------------------------------------------------------------")
print("default 2")
list_default_docs2 = []
# Garantir que temos ao menos 3 elementos
n = len(default_docs2)
first_n = default_docs2[:3]            # primeiros 3
last_n = default_docs2[-3:] if n >= 3 else []  # últimos 3 (sem repetir os primeiros)
# Adiciona os page_content dos primeiros
for i, doc in enumerate(first_n):
    print(f"Primeiro [{i}] METADATA =>", doc.metadata)
    list_default_docs2.append(doc.page_content)
# Adiciona os page_content dos últimos
for i, doc in enumerate(last_n):
    print(f"Último [{n - 3 + i}] METADATA =>", doc.metadata)
    list_default_docs2.append(doc.page_content)
print("-----------------------------------------------------------------------------------------")

In [ ]:
for i, doc in enumerate(list_default_docs):
    print(f" {i}:  {doc} ...")
print("-----------------------------------------------------------------------------------------")

for i, doc in enumerate(list_default_docs2):
    print(f" {i}:  {doc} ...")
print("-----------------------------------------------------------------------------------------")

In [ ]:
try:
    openai_emb = cc.EmbeddingOpeanAI(
        list_default_docs, 
        API_KEY,
    )
    print(f"[EmbeddingOpeanAI] Embedding gerada com dim={len(openai_emb[0])}")
    print(type(openai_emb))
    for i, item in enumerate(openai_emb):
        print(f"{i}:")
        print(type(item))
        print(item)

    # (c) Normalizar a primeira embedding
    print("\n\n\n")
    cc.NormalizeEmbeddings(openai_emb[0])
except Exception as e:
    print("Não foi possível gerar embeddings com OpenAI (talvez sem chave válida):", e)
print()

In [ ]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# 3) CHUNKANDO COM ChunkCount (divide texto a cada pontuação final, usando regex, overlap menor)
# ChunkCount(const std::string &count_unit, const int overlap = 600, const int count_threshold = 1);
count_chunker = cc.ChunkCount("regex:[.!?]+", overlap=10, count_threshold=1)
count_chunker2 = cc.ChunkCount("regex:[.!?]+", overlap=10, count_threshold=1)

In [ ]:
# std::vector<RAGLibrary::Document> ProcessData(RAGLibrary::LoaderDataStruct &item);
count_docs = count_chunker.ProcessData(item_struct[0])
# std::vector<RAGLibrary::Document> ProcessData(const std::vector<RAGLibrary::LoaderDataStruct> &items, int max_workers = 4);
count_docs2 = count_chunker2.ProcessData(item_struct, max_workers=5) 

In [ ]:
print(f"ChunkCount from {len(count_docs)}")
print(f"ChunkCount2 from {len(count_docs2)}")

In [ ]:

print("-----------------------------------------------------------------------------------------")
print("Count")
list_count_docs = []
prev_m = None
for i, doc in enumerate(count_docs):
    if i > 3: break
    if doc.metadata != prev_m or prev_m is None:
        print(" METADATA =>", doc.metadata)
    print(type(doc))
    print(type(doc.page_content))
    list_count_docs.append(doc.page_content)
    print(f"{i}:  {doc}")
    prev_m = doc.metadata

print("-----------------------------------------------------------------------------------------")
print("Count 2")
list_count_docs2 = []
prev_m = None
for i, doc in enumerate(count_docs2):
    if i > 3: break
    if doc.metadata != prev_m or prev_m is None:
        print(" METADATA =>", doc.metadata)
    print(type(doc))
    print(type(doc.page_content))
    list_count_docs2.append(doc.page_content)
    print(f"{i}:  {doc}")
    prev_m = doc.metadata
print("-----------------------------------------------------------------------------------------")

In [ ]:
for i, doc in enumerate(list_count_docs):
    print(f" {i}:  {doc[:70]} ...")
print("-----------------------------------------------------------------------------------------")

for i, doc in enumerate(list_count_docs2):
    print(f" {i}:  {doc[:70]} ...")
print("-----------------------------------------------------------------------------------------")

In [ ]:
# 4) USANDO ALGUMAS FUNÇÕES DE CHUNKCOMMONS MANUALMENTE
#    Exemplo:  (a) Dividir manualmente um texto e (b) Gerar embeddings via OpenAI
#    (c) Normalizar e exibir a embedding

# (a) SplitText manual
text_example = ["Este é um texto de exemplo.", "Ele será unido e fatiado em 50 chars com 10 de overlap."]
splitted = cc.SplitText(text_example, overlap=10, chunk_size=50)
print(f"[SplitText] Gerei {len(splitted)} pedaços. Primeiro pedaço:\n   {splitted[0]}")

In [ ]:
# 5) USANDO FUNÇÕES DE EMBEDDINGS MANUALMENTE
#    Exemplo:  (a) Gerar embeddings via OpenAI
#    (b) Normalizar e exibir a embedding
try:
    openai_emb = cc.EmbeddingOpeanAI(
        list_count_docs, # tem q ser uma lista de string
        API_KEY,
    )
    print(f"[EmbeddingOpeanAI] Embedding gerada com dim={len(openai_emb[0])} ")
    for i, item in enumerate(openai_emb):
        print(f"{i}:")
        print(type(item))
        print(item)

    cc.NormalizeEmbeddings(openai_emb[0])
except Exception as e:
    print("Não foi possível gerar embeddings com OpenAI (talvez sem chave válida):", e)
print()